# Convolutional Neural Network

### Importing the libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import os
import random

In [ ]:
tf.__version__

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('Vegetable_Images/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('Vegetable_Images/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
training_set[0]

## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=15, activation='softmax'))

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 15)

## Part 4 - Making a single prediction

In [ ]:
# Categories corresponding to the class labels
categories = ['Bean', 'Bitter_Gourd', 'Bottle_Gourd', 'Brinjal', 'Broccoli', 'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Cucumber', 'Papaya', 'Potato', 'Pumpkin', 'Radish', 'Tomato']

# Function to load a random image from the test set
def load_random_image(test_image_directory):
    # Randomly pick a category folder (ant, bee, cockroach, spider)
    random_class = random.choice(os.listdir(test_image_directory))
    random_image_folder = os.path.join(test_image_directory, random_class)
    
    # Randomly pick an image from the selected category
    random_image_filename = random.choice(os.listdir(random_image_folder))
    random_image_path = os.path.join(random_image_folder, random_image_filename)
    
    # Load and preprocess the image
    test_image = image.load_img(random_image_path, target_size=(64, 64))
    test_image_array = image.img_to_array(test_image)
    test_image_array = np.expand_dims(test_image_array, axis=0)  # Add batch dimension
    test_image_array = test_image_array / 255.0  # Normalize pixel values

    return test_image_array, random_image_path  # Return image array and path to display

# Set directory for test images
test_image_directory = 'Vegetable_Images/test'

# Load a random image from the test set
test_image_array, random_image_path = load_random_image(test_image_directory)

# Make prediction
result = cnn.predict(test_image_array)

# Find the index of the class with the highest probability
predicted_class_index = np.argmax(result, axis=1)[0]

# Get the corresponding category label
predicted_category = categories[predicted_class_index]

# Display the image
plt.figure(figsize=(3, 3))  # Set the figure size (width, height)
img = image.load_img(random_image_path)
plt.imshow(img)
plt.axis('off')  # Turn off axis labels
plt.show()

# Print the predicted category
print(f"The predicted category is: {predicted_category}")


In [ ]:
# Plotting Training & Validation Loss
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()